Importing libraries


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import sqlite3
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium import webdriver
from multiprocessing.pool import ThreadPool as Pool
import multiprocessing
import threading
from time import sleep

Setting up a fuction to work along the scrap


In [4]:
class OLX:
        def __init__(self, queue, lock):
            self.queue = queue
            self.lock = lock
            self.all_data = []
            self.finish_first = False  

            options = Options()
            options.add_argument('--headless')

            self.driver = webdriver.Firefox(options=options)
        def requests_olx(self, url):
            self.driver.get(url)
            return self.driver.page_source
        
        def anuncios_pg(self, url):
            pg = 0

            while True:
                pg+=1
                print('PAGE: ', pg)
                content_pg = self.requests_olx(url)
                source_pg = BeautifulSoup(content_pg, 'html.parser')
                list_anun = list(map(lambda x: x.attrs['href'], source_pg.find_all('a', attrs={'data-ds-component':'DS-NewAdCard-Link'})))
                self.lock.acquire()   
                [self.queue.put(i) for i in list_anun]
                self.lock.release()
                self.finish_first = True


                if source_pg.select_one('a[data-ds-component="DS-Button"].olx-button.olx-button--link-button.olx-button--small.olx-button--a:nth-of-type(12)') or pg > 5:
                    url = source_pg.select_one('a[data-ds-component="DS-Button"].olx-button.olx-button--link-button.olx-button--small.olx-button--a:nth-of-type(12)').get('href')

                else:
                    print('acabou de pegar os links...')
                    break
                if pg > 3:
                    break

        def anuncios(self, id_robot=None, url=None):
            while True:
                 sleep(1)
                 if self.finish_first == True:
                    url = self.queue.get(block = True)
                    break
            while True:


                content_pg = self.requests_olx(url)
                source_pg = BeautifulSoup(content_pg, 'html.parser')
                values = {}
                try:
                    title_span = source_pg.find('span', class_='olx-text--title-medium')
                    values['title'] =  title_span.text.strip() if title_span else None

                    owner_name_span = source_pg.find('span', class_='olx-text olx-text--body-large olx-text--block olx-text--regular ad__sc-ypp2u2-4 TTTuh')
                    values['owner_name'] = owner_name_span.text if owner_name_span else None

                    description_span = source_pg.find('meta', {'property':'og:description'})
                    values['description'] = description_span.get('content') if description_span else None

                    url_anuncio = source_pg.find('meta', {'property':'og:url'})
                    url_id = url_anuncio.get('content') if url_anuncio else None
                                
                    values['id'] = url_id.split('-')[-1]

                    adress_span = source_pg.find('span', class_='olx-text olx-text--body-small olx-text--block olx-text--semibold olx-color-neutral-110')
                    adress_text = adress_span.text.strip() if adress_span else None
                    v = adress_text.split(',') if adress_text else None
                    values['city'] = v[0].strip() if adress_text else None
                    values['state'] = v[1].strip() if adress_text else None
                    values['cep'] = v[2].strip() if adress_text else None

                except:
                    print("***",url)
                    break

                print('Proc_id: {} | link: {}'.format(id_robot, url))

                self.lock.acquire()   
                self.all_data.append(values)
                self.lock.release()
                
                if self.queue.qsize() == 0:
                    break
                else:
                    url = self.queue.get(block = True)
                

            
           
        
        def __del__(self):
            self.driver.quit()

        


In [8]:

num_robot = 4
pool = Pool(num_robot)

manager = multiprocessing.Manager()
queue = manager.Queue()
lock = multiprocessing.Lock()

threads = []
olx = OLX(queue, lock)

add_itens = threading.Thread(target=olx.anuncios_pg, args=('https://www.olx.com.br/servicos?sct=9',))
add_itens.start()

for i in range(1,num_robot):
    threads.append(pool.apply_async(olx.anuncios, args=(i, None)))
for result in threads:
    result.get()
pool.close()
pool.join()
add_itens.join()


PAGE:  1
PAGE:  2
acabou de pegar os links...
Proc_id: 2 | link: https://go.olx.com.br/grande-goiania-e-anapolis/servicos/venda-de-terra-aterros-demolicoes-retirada-de-entulhos-lopes-locacoes-1089033240
Proc_id: 3 | link: https://rn.olx.com.br/rio-grande-do-norte/servicos/frete-natal-e-pequenas-mudancas-apartir-1272488517
*** https://rn.olx.com.br/rio-grande-do-norte/servicos/frete-natal-e-pequenas-mudancas-apartir-1272488517
*** https://go.olx.com.br/grande-goiania-e-anapolis/servicos/venda-de-terra-aterros-demolicoes-retirada-de-entulhos-lopes-locacoes-1089033240
Proc_id: 3 | link: https://ce.olx.com.br/fortaleza-e-regiao/servicos/despachante-documentalista-agente-credenciado-1167619849
Proc_id: 3 | link: https://ce.olx.com.br/fortaleza-e-regiao/servicos/despachante-documentalista-agente-credenciado-1167619849
Proc_id: 3 | link: https://ce.olx.com.br/fortaleza-e-regiao/servicos/eletricista-bombeiro-banheira-de-hidro-coifa-ar-condicionado-geladeira-antena-coletiva-865535065
Proc_id: 3

In [15]:
df = pd.DataFrame(olx.all_data)
df

,title,owner_name,description,id,city,state,cep
0,Frete natal e pequenas mudanças apartir,Izaura Sil,Pequenas mudanças preço a combinar Ligue e com...,1272488517,Natal,RN,59030120
1,Frete natal e pequenas mudanças apartir,Izaura Sil,Pequenas mudanças preço a combinar Ligue e com...,1272488517,Natal,RN,59030120
2,Despachante Documentalista Agente Credenciado,None,* 1° Emplacamento <br>* 2° Via de Documento<br...,1167619849,Fortaleza,CE,60712111
3,Despachante Documentalista Agente Credenciado,None,* 1° Emplacamento <br>* 2° Via de Documento<br...,1167619849,Fortaleza,CE,60712111
4,"Eletricista/Bombeiro, Banheira de Hidro, Coifa...",None,"Fonezap: 99951-9308 Eletricista, Bombeiro, Ins...",865535065,Fortaleza,CE,60150162
5,"Eletricista/Bombeiro, Banheira de Hidro, Coifa...",None,"Fonezap: 99951-9308 Eletricista, Bombeiro, Ins...",865535065,Fortaleza,CE,60150162
6,montador de móveis,None,Desmontagem e Montagem <br>Whatsapp 98214-9392...,1273268462,Goiânia,GO,74360360
7,montador de móveis,None,Desmontagem e Montagem <br>Whatsapp 98214-9392...,1273268462,Goiânia,GO,74360360


In [18]:
import sqlite3 as sq

sql_data = r'C:\Users\estev\OneDrive\Desktop\projetos-estevao\OLX_SCRAP\OLX_BD_SQLITE.sqlite'

In [19]:

conn = sq.connect(sql_data)
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS SA''')


In [20]:
df.to_sql('imoveis', conn, if_exists='replace', index=False)

8

In [21]:
df_new = pd.read_sql_query('SELECT * FROM imoveis', conn)
df_new

,title,owner_name,description,id,city,state,cep
0,Frete natal e pequenas mudanças apartir,Izaura Sil,Pequenas mudanças preço a combinar Ligue e com...,1272488517,Natal,RN,59030120
1,Frete natal e pequenas mudanças apartir,Izaura Sil,Pequenas mudanças preço a combinar Ligue e com...,1272488517,Natal,RN,59030120
2,Despachante Documentalista Agente Credenciado,None,* 1° Emplacamento <br>* 2° Via de Documento<br...,1167619849,Fortaleza,CE,60712111
3,Despachante Documentalista Agente Credenciado,None,* 1° Emplacamento <br>* 2° Via de Documento<br...,1167619849,Fortaleza,CE,60712111
4,"Eletricista/Bombeiro, Banheira de Hidro, Coifa...",None,"Fonezap: 99951-9308 Eletricista, Bombeiro, Ins...",865535065,Fortaleza,CE,60150162
5,"Eletricista/Bombeiro, Banheira de Hidro, Coifa...",None,"Fonezap: 99951-9308 Eletricista, Bombeiro, Ins...",865535065,Fortaleza,CE,60150162
6,montador de móveis,None,Desmontagem e Montagem <br>Whatsapp 98214-9392...,1273268462,Goiânia,GO,74360360
7,montador de móveis,None,Desmontagem e Montagem <br>Whatsapp 98214-9392...,1273268462,Goiânia,GO,74360360
